<a href="https://colab.research.google.com/github/Ericdelucas/-Ericdelucas-Untitled0/blob/main/C%C3%B3pia_de_Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install python-Levenshtein
!pip install sentence-transformers
!pip install tk

In [38]:
import requests
from sentence_transformers import SentenceTransformer, util
from collections import OrderedDict
import numpy as np


url = "https://imdb146.p.rapidapi.com/v1/find/"


headers = {
    "X-RapidAPI-Key": "3dee243514mshf52bd8de7ff75a3p16eb03jsn626f07172a07",
    "X-RapidAPI-Host": "imdb146.p.rapidapi.com"
}


modelo_sbert = SentenceTransformer('paraphrase-MiniLM-L6-v2')


historico_pesquisas = OrderedDict()


maximo_historico = 5 

contador_pesquisas = 0


def realizar_pesquisa(consulta):
    global contador_pesquisas
    
    
    if consulta in historico_pesquisas:
        print(f"Resultados para '{consulta}' já estão armazenados em histórico.")
        return historico_pesquisas[consulta]

    
    response = requests.get(url, headers=headers, params={"query": consulta})

    if response.status_code == 200:
        json_response = response.json()
        consulta_vector = modelo_sbert.encode(consulta)
        title_results = json_response.get("titleResults")
        
        if title_results and title_results.get("results"):
            resultados_com_similaridade = []

            for filme in title_results["results"]:
                title_name = filme.get("titleNameText")
                if title_name:
                    title_vector = modelo_sbert.encode(title_name)
                    similaridade = util.pytorch_cos_sim(consulta_vector, title_vector).item()
                    resultados_com_similaridade.append((similaridade, filme))

            resultados_com_similaridade.sort(reverse=True, key=lambda x: x[0])
            resultados_com_similaridade = resultados_com_similaridade[:3]

            historico_pesquisas[consulta] = resultados_com_similaridade
            
            if len(historico_pesquisas) > maximo_historico:
                historico_pesquisas.popitem(last=False)
            
            contador_pesquisas += 1  # Incrementar o contador
            
            resultados_text = ""
            for i, (similaridade, filme) in enumerate(resultados_com_similaridade, start=1):
                title_name = filme.get("titleNameText")
                release_text = filme.get("titleReleaseText")
                title_type = filme.get("titleTypeText", "Não disponível")
                poster_url = filme.get("titlePosterImageModel", {}).get("url")
                similaridade_ajustada = similaridade * 99.9
                
                resultados_text += (f"Resultado #{i}:\n"
                                    f"Nome do filme: {title_name}\n"
                                    f"Data de lançamento: {release_text}\n"
                                    f"Categorias: {title_type}\n"
                                    f"URL da imagem do filme: {poster_url}\n"
                                    f"Similaridade: {similaridade_ajustada:.2f}%\n\n")
            
            print(resultados_text)
        else:
            print(f"Nenhum filme encontrado para o termo de busca: {consulta}")
    else:
        print(f"Erro na requisição: {response.status_code}")

# Programa principal
if __name__ == "__main__":
    while True:
        print(f"\nMenu: (Pesquisas realizadas: {contador_pesquisas})")
        print("1. Realizar uma pesquisa")
        print("2. Sair")
        
        opcao = input("Escolha uma opção (1/2): ")
        
        if opcao == "1":
            consulta = input("Digite o termo de busca: ")
            realizar_pesquisa(consulta)
        elif opcao == "2":
            print("Saindo...")
            break
        else:
            print("Opção inválida. Tente novamente.")



Menu:
1. Realizar uma pesquisa
2. Obter recomendações com base nas últimas pesquisas
3. Sair
Escolha uma opção (1/2/3): gato de botas
Opção inválida. Tente novamente.

Menu:
1. Realizar uma pesquisa
2. Obter recomendações com base nas últimas pesquisas
3. Sair
Escolha uma opção (1/2/3): 1
Digite o termo de busca: gato de botas
Resultado #1:
Nome do filme: O Gato de Botas Extraterrestre
Data de lançamento: 1990
Categorias: 
URL da imagem do filme: https://m.media-amazon.com/images/M/MV5BYTFiM2EzNTgtNjlhOS00NDEwLWI5MjUtZDZhYTdjN2NhNTM1XkEyXkFqcGdeQXVyNDUxNjc5NjY@._V1_.jpg
Similaridade: 83.67%

Resultado #2:
Nome do filme: Puss in Boots
Data de lançamento: 2011
Categorias: 
URL da imagem do filme: https://m.media-amazon.com/images/M/MV5BMWZiZTc0NTktNzNmZC00M2M3LTkwMDMtZGU3Mzg4NWI0ZjdjXkEyXkFqcGdeQXVyNTgyNTA4MjM@._V1_.jpg
Similaridade: 19.06%

Resultado #3:
Nome do filme: Puss in Boots
Data de lançamento: 1988
Categorias: 
URL da imagem do filme: https://m.media-amazon.com/images/M/MV5BYT

/usr/local/lib/python3.10/dist-packages/sentence_transformers/util.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  b = torch.tensor(b)


Recomendações com base na nova palavra gerada:
Nome do filme: The Words
Data de lançamento: 2012
Categorias: 
URL da imagem do filme: https://m.media-amazon.com/images/M/MV5BMTM2NjgyMjI3OV5BMl5BanBnXkFtZTcwMDkxMjIyOA@@._V1_.jpg

Nome do filme: Mr. Smith Goes to Washington
Data de lançamento: 1939
Categorias: 
URL da imagem do filme: https://m.media-amazon.com/images/M/MV5BZTYwYjYxYzgtMDE1Ni00NzU4LWJlMTEtODQ5YmJmMGJhZjI5L2ltYWdlXkEyXkFqcGdeQXVyMDI2NDg0NQ@@._V1_.jpg

Nome do filme: Ordet
Data de lançamento: 1955
Categorias: 
URL da imagem do filme: https://m.media-amazon.com/images/M/MV5BYjFjYzQ4ZTgtZGQ3MS00NDE2LTlmOTktYTIwOWIwMWVkYjcxL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyNTc2MDU0NDE@._V1_.jpg

Nome do filme: Don't Say a Word
Data de lançamento: 2001
Categorias: 
URL da imagem do filme: https://m.media-amazon.com/images/M/MV5BYmZmNTg4NGQtOTMxZi00MWU3LTlkY2MtMjEyNzk3NTBlNzk0XkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_.jpg

Nome do filme: The Garden of Words
Data de lançamento: 2013
Categorias: 
URL da ima